Estimators 封装在下列actions当中：
<ul type='disc'>
    <li>training</li>
    <li>evaluation</li>
    <li>prediction</li>
    <li>export for serving</li>
</ul>
<font size="2" color="grey"> 所有的Estimators都基于tf.estimator.Estimators 类(tf.contrib.learn.Estimator)已经被废弃，不应该继续使用</font>   

<b> Estimator 优势：</b>

<ul>
    <li>使用Estimator_based模型，不需要在不同的平台上修改代码</li>
    <li>Estimators simplify sharing implementations between model developers</li>
    <li>You can develop a state of the art model with high-level intuitive code, In short, it is generally much easier to create models with Estimators than with the low-level TensorFlow APIs.</li>
    <li>Estimators 简历在tf.layers上面，可以简单的被自定义</li>
    <li>Estimators可以自动的创建Graph</li>
    <li>Estimators可以提供safe distributed training loop that controls how and when to:</li>
    <ul>
    <li>构建图形</li>
    <li>初始化变量</li>
    <li> start queues</li>
    <li>处理已成</li>
    <li>create checkpoint files and recover from failures</li>
    <li>为Tensorboard保存summaries</li>
    </ul>
    </ul>  
当写一个具有Estimators的应用是，you must separate the data input pipeline from the model.这样做可便于实验数据的替换。


# Pre-made Estimators
pre-made Estimators可以创建和管理Graph和Session对象。   
Furthermore, pre-made Estimators let you experiment with different model architectures by making only minimal code changes. DNNClassifier, for example, is a pre-made Estimator class that trains classification models through dense, feed-forward neural networks.

<b>tf.estimator.LinearClassifier:</b> Constructs a linear classification model.
<b>tf.estimator.LinearRegressor:</b> Constructs a linear regression model.  
<b>tf.estimator.DNNClassifier:</b> Construct a neural network classification model.  
<b>tf.estimator.DNNRegressor:</b> Construct a neural network regression model.  
<b>tf.estimator.DNNLinearCombinedClassifier:</b> Construct a neural network and linear combined classification model.  
<b>tf.estimator.DNNLinearCombinedRegressor:</b> Construct a neural network and linear combined regression model.  

### Structure of a pre-made Estimators program
1.<b> 写一个或者多个dataset导入函数。</b>  
input_fn
<p> 每个dataset导入函数返回两个对象：
    <ul>
        <li> 字典：keys：特征名称，values：Tensor(or sparse Tensros)对应相应的特征。</li>
        <li> Tensor: 包含一个或者多个标签</li>
    </ul>
</p>
<p>
    input function(input_fn)的主题包含了input data预处理过程，必须要返回两个值包含处理过的feature和label(用于传递给模型)：（函数的skeleton见上面)my_input_fn):  
    <ul>
        <li><b>feature_cols: </b>dict: 包含key/value成对对应到feature columns names和包含相应特征的Tensor(Sparse Tensor)</li>
        <li><b>labels: </b>a Tensor包含目标值(label)，即模型的预测值</li>
    </ul>
</p>
<p>
    如果feature/label data使用python array存储在pandas dataframe或者numpy array中，可以用以下方法来构建input_fn:详细例子如下
</p>
<p>
    对于sparse,categorical data(大部分值为0),那么input_fn可以改为填充一个SparseTensor，可以通过三个参数实例化：
    <ul>
        <li><b>dense_shape: </b>  
            the shape of tensor.list类型，每一个元素代表当前的维度，e.g.dense_shape=[3,6],[2,3,4],[9]分别代表二维，三维，一维的tensor
        </li>
        <li><b>indices: </b>  
            知名非零值出现的位置，list类型。index以0开始，即[0,0]表示第一行第一列，indices=[[1,3],[2,4]]代表[1,3],[2,4]的index处不为零
        </li>
        <li><b>values: </b>  
            一维tensor，第i个值表示indices中的第i个元素填充的值
        </li>
    </ul>
</p>

In [5]:
import tensorflow as tf
#e.g.: 
def input_fn(dataset):
    ...# manipulate dataset, extracting feature names and the label
    return feature_dict,label

def my_input_fn():
    # Preprocess your data here...

    # ...then return 1) a mapping of feature columns to Tensors with
    # the corresponding feature data, and 2) a Tensor containing labels
    return feature_cols, labels

import numpy as np
# numpy input_fn.
my_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_data)},
    y=np.array(y_data),
    ...)

import pandas as pd
# pandas input_fn.
my_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=pd.DataFrame({"x": x_data}),
    y=pd.Series(y_data),
    ...)



In [22]:
sparse_tensor = tf.SparseTensor(indices=[[0,1], [2,4]],
                                values=[6, 0.5],
                                dense_shape=[3, 5])
with tf.Session() as sess:
    print(sess.run(sparse_tensor).dense_shape)

[3 5]


2.<b> 定义特征columns。</b>每一个tf.feature_column定义(identifies)一个特征名字，他的类型和任何的预处理（input pre-processing)  
<p>这个例子中创建了三个特征columns包含integer或者是float-point数据
</p>


In [8]:
# Define three numeric feature columns
population = tf.feature_column.numeric_column('population')
crime_rate = tf.feature_column.numeric_column('crime_rate')
median_education = tf.feature_column.numeric_column('median_edcation',
                                                     normalizer_fn='lambda x:x-global_education_mean')

3.<b> 实例化相关的pre-made Estimator。</b>
<p>这个例子中实例化pre-made Estimator为LinearClassifier
</p>

In [10]:
# Instantiate an estimator, passing the feature columns.
estimator = tf.estimator.Estimator.LinearClassifier(
    feature_columns=[population,crime_rate,median_education],)

4.<b> Call a training, evaluation, or inference method。</b>
<p>所有的Estimator都提供一个train模型
</p>

In [ ]:
# my_training_set is the function created in Step 1
estimator.train(input_fn=my_training_set,steps=2000)

## Benefits of pre-made Estimators
Pre-made Estimators encode best practices, providing the following benefits:  
<ul>
    <li>Best practices for determining where different parts of the computational graph should run, implementing strategies on a single machine or on a cluster.</li>
    <li>Best practices for event (summary) writing and universally useful summaries.</li>
</ul>
如果不适用pre-made Estimators,那么需要自行实践preceding features
    

In [ ]:
## pre-made Estimator 实例：DNNclassifier，实行dense，feed-forward NN

categorical_feature_a = categorical_column_with_hash_bucket(...)
categorical_feature_b = categorical_column_with_hash_bucket(...)

categorical_feature_a_emb = embedding_column(
    categorical_column=categorical_feature_a, ...)
categorical_feature_b_emb = embedding_column(
    categorical_column=categorical_feature_b, ...)

estimator = DNNClassifier(
    feature_columns=[categorical_feature_a_emb, categorical_feature_b_emb],
    hidden_units=[1024, 512, 256])

# Or estimator using the ProximalAdagradOptimizer optimizer with
# regularization.
estimator = DNNClassifier(
    feature_columns=[categorical_feature_a_emb, categorical_feature_b_emb],
    hidden_units=[1024, 512, 256],
    optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.1,
      l1_regularization_strength=0.001
    ))

# Input builders
def input_fn_train: # returns x, y
    pass
    #e.g.  
     return tf.estimator.inputs.pandas_input_fn(
      x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
      y = pd.Series(data_set[LABEL].values),
      num_epochs=num_epochs,
      shuffle=shuffle)
estimator.train(input_fn=input_fn_train, steps=100)

def input_fn_eval: # returns x, y
    pass
metrics = estimator.evaluate(input_fn=input_fn_eval, steps=10)
def input_fn_predict: # returns x, None
    pass
predictions = estimator.predict(input_fn=input_fn_predict)

# Custom Estimators
每一个Estimator的核心（不管是pre-made还是custom）都是model function，
是一个建立training, evaluation, 和prediction的方法。  
当用 pre-made Estimator的时候，别人已经implemented the model function。
在custom Estimator的时候，必须要自己写一个方程  
步骤：
<ul>
    <li>Instantiate an Estimator</li>
    <li>Construct a custom model function</li>
    <li>Configure a neural network using tf.feature_column and tf.layers</li>
    <li>Choose an appropriate loss function from tf.losses</li>
    <li>Define a training op for your model</li>
    <li>Generate and return predictions</li>
</ul>

nn = tf.estimator.Estimator(model_fn=model_fn, params=model_params)
<ul>
    <li><b>model_fn:</b> A function object that contains all the aforementioned logic to support training, evaluation, and prediction. You are responsible for implementing that functionality. The next section, Constructing the model_fn covers creating a model function in detail.</li>

<li><b>params:</b> An optional dict of hyperparameters (e.g., learning rate, dropout) that will be passed into the model_fn.</li>
</ul>

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile

# Import urllib
from six.moves import urllib

import numpy as np
import tensorflow as tf

FLAGS = None
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
def maybe_download(train_data, test_data, predict_data):
    """Maybe downloads training data and returns train and test file names."""
    if train_data:
        train_file_name = train_data
    else:
        train_file = tempfile.NamedTemporaryFile(delete=False)
        urllib.request.urlretrieve(
            "http://download.tensorflow.org/data/abalone_train.csv",
            train_file.name)
        train_file_name = train_file.name
        train_file.close()
        print("Training data is downloaded to %s" % train_file_name)

    if test_data:
        test_file_name = test_data
    else:
        test_file = tempfile.NamedTemporaryFile(delete=False)
        urllib.request.urlretrieve(
            "http://download.tensorflow.org/data/abalone_test.csv", test_file.name)
        test_file_name = test_file.name
        test_file.close()
        print("Test data is downloaded to %s" % test_file_name)

    if predict_data:
        predict_file_name = predict_data
    else:
        predict_file = tempfile.NamedTemporaryFile(delete=False)
        urllib.request.urlretrieve(
            "http://download.tensorflow.org/data/abalone_predict.csv",
            predict_file.name)
        predict_file_name = predict_file.name
        predict_file.close()
        print("Prediction data is downloaded to %s" % predict_file_name)

    return train_file_name, test_file_name, predict_file_name

In [ ]:
# instantiating an estimator
# pre-made estimator
my_nn = tf.estimator.DNNClassifier(feature_columns=[age, height, weight],
                                   hidden_units=[10, 10, 10],
                                   activation_fn=tf.nn.relu,
                                   dropout=0.2,
                                   n_classes=3,
                                   optimizer="Adam")

In [ ]:
# custom estimator
nn = tf.estimator.Estimator(model_dn=model_fn,params=model_params)# 两个参数
def model_fn(features, labels, mode, params):
   # Logic to do the following:
   # 1. Configure the model via TensorFlow operations
   # 2. Define the loss function for training/evaluation
   # 3. Define the training operation/optimizer
   # 4. Generate predictions
   # 5. Return predictions/loss/train_op/eval_metric_ops in EstimatorSpec object
    return EstimatorSpec(mode, predictions, loss, train_op, eval_metric_ops)

def my_input_fn():

    # Preprocess your data here...

    # ...then return 1) a mapping of feature columns to Tensors with
    # the corresponding feature data, and 2) a Tensor containing labels
    return feature_cols, labels

model_fn接收三个参数：
<ul>
    <li><b>feature: </b> dict：包含特征，通过input_fn传递给model</li>
    <li><b>labels: </b>Tensor：包含labels, 通过input_fn传递给model</li>
    <li><b>mode: </b>tf.estimator.ModeKeys的string值代表会工作一下那个model_fn:</li>
<p>
    input_fn见最上面的说明
</p>
    <ul>
        <li> tf.estimator.ModeKeys.TRAIN：model_fn执行training mode，通过train()调用</li>
        <li> tf.estimator.ModeKeys.EVAL: model_fn执行evaluation mode,通过evaluate()调用</li>
        <li> tf.estimator.ModeKeys.PREDICT: model_fn执行predict mode，通过predict()调用</li>
    </ul>
</ul>
model_fn 也可以接收params参数（包含一个hyperparameters的字典），用来训练  
函数的主体执行以下功能：
<ul>
    <li>配置模型:集模型的结构</li>
    <li>定义损失函数</li>
    <li>定义训练的操作，和optimizer的算法来让损失函数变小</li>
</ul>
<b>return:</b>  
返回tf.estimator.EstimatorSpec 对象，包含以下值：  
__new__(
    cls,  
    mode,  
    predictions=None,  
    loss=None,  
    train_op=None,  
    eval_metric_ops=None,  
    export_outputs=None,  
    training_chief_hooks=None,  
    training_hooks=None,  
    scaffold=None,  
    evaluation_hooks=None  
)
<ul>
    <li><b>mode(required) </b>这个model运行的mode，一般而言会在model_fn中返回mode参数</li>
    <ul>
        <li>For <b>mode == ModeKeys.TRAIN: </b>required fields are loss and train_op.</li>
        <li>For <b>mode == ModeKeys.EVAL: </b>required field is loss.</li>
        <li>For <b>mode == ModeKeys.PREDICT: </b>required fields are predictions.</li>
    </ul>
    <li><b>predictions(required in PREDICT model) </b>dict：将选择的名字于model预测值得Tensor匹配 
        e.g. python predictions = {"results":tensor_of_predictions}  
        PREDICT mode,在EstimatorSpec中得dict会通过predict()返回
    </li>
    <li><b>loss(required in EVAL and TRAIN mode) </b>A Tensor 包含一个标量损失值</li>
    <li><b> train_op (required only in TRAIN mode) </b>An Op that runs one step of training.</li>
    <li><b> eval_metric_ops (optional) </b>dict: name/value匹配特定得metrics可以被模型在EVAL模型中计算，name是选择得metric，value是计算结果，tf.metrics模块提供一些普遍的已经被定义好得函数  
        e.g. 一下的评估中包含一个‘accuracy'的metric使用tf.metrics.accuracy计算：  
      python eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels,predictions)}  
        如果不用这个参数，那么在evaluation的时候只有loss会被计算
    </li>        


### Configuring a neural network with tf.feature_column and tf.layers
<b>model construction +predictions</b>  
neural network构建包含三个部分：input layer, hidden layers和output layer  

input layer：包含一些列的nodes接收feature data(通过features参数传递到model_fn中)当features包含n-dimension Tensor(所有的特征数据),那么这层就作为input layer。  
features： dict of feature_columns，通过input function传递到model中，可以通过tf.feature_column.input_layer函数将其转换成input_layer Tensor。  
e.g. : input_layer = tf.feature_column.input_layer(features=features,feature_columns=[age, height, weight])  
input_layer()包含两个required参数：
<ul>
    <li><b>features </b>a mapping 从字符串keys到包含相对应的feature data Tensor。这就是传递到model_fn中的feature参数</li> 
    <li><b>feature_columns </b>a list 包含所有的模型中的FeatureColumns，例如以上例子中的age, height, weight</li>
</ul>
<p>
    input layer通过activation function(对前一层输出的数据进行非线性转换)和hidden layers连接，hidden layer的最后一层连接到输出层，model的最后一层，tf.layers提供tf.layers.dense方程来构建全连接层。activation通过activation参数进行控制。activation其中一些选择有：
    <ul>
        <li><b>tf.nn.relu </b>e.g. 创建layer units nodes全连接到前一层的input_layer用relu：  
            python hidden_layer= tf.layers.dense(inputs=input_layer,units=10,activation=tf.nn.relu)
        </li>
        <li><b>tf.nn.relu6 </b>计算方法：min(max(features, 0), 6)  
            e.g. python second_hidden_layer = tf.layers.dense( inputs=hidden_layer, units=20, activation=tf.nn.relu)
        </li>
        <li><b>None </b> 不连接activate function，只有linear transformation  
            python output_layer = tf.layers.dense( inputs=second_hidden_layer, units=3, activation=None)
        </li>
    </ul>
其他可能选择：  
output_layer = tf.layers.dense(inputs=second_hidden_layer,
                               units=10,
                               activation_fn=tf.sigmoid)  
上面的code船舰了一个output_layer，通过一个sigmoid激活函数全连接到second_hidden_layer
</p>

### Defining loss for the model
model_fn返回的EstimatorSpec必须要包含loss：a Tensor(表示loss值)  
tf.losses模块提供convenience functions计算不同metrics中的loss：
<ul>
    <li><b>absolute_difference(labels, predictions) </b>计算loss用absolute-difference方程计算(L1 loss)</li>
    <li><b>log_loss(labels, predictions) </b>计算loss用logistic loss formula计算(一般用logistic regression)</li>
    <li><b>mean_squared_error(labels, predictions) </b>(MSE;用L2 loss)</li>
</ul>
以下例子中的model_fn用mean_squared_error()计算  
<p>
    补充的用来evaluation的metrics加到eval_metric_ops字典当中，以下code定义rmse metric，用来在model predictions时计算root mean sqaured error。labels tensor需要转换到float64来匹配predictions tensor的数据类型（包含实数）：  
    eval_metric_ops = {"rmse":tf.metrics.root_mean_squared_error(  
         tf.cast(lables,tf.float64),predictions))

### Defining the training op for the model
training op定义优化算法。一般而言，在训练的时候，目标是最小化loss。一个简单的来创建training op的方法是实例化tf.train.Optimizer的子类并且调用它的minimize方法
<p>
    以下用例当中learning_rate传送到function的param参数当中，使用gradient descent optimizer。对于global_step，采用convenience function tf.train.get_global_step来生成一个整数变量：  
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=params["learning_rate"])  
    training_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    </p>

In [ ]:
## 完整的model_fn
def model_fn(features,labels,mode,params):
    """Model function for Estimator."""

  # Connect the first hidden layer to input layer
  # (features["x"]) with relu activation
    first_hidden_layer = tf.layer.dense(feature['x'],10,activation=tf.nn.relu)
    
    # Connect the second hidden layer to first hidden layer with relu
    second_hidden_layer = tf.layers.dense(first_hidden_layer,10,activation=tf.nn.relu)
    
    # Connect the output layer to second hidden layer (no activation fn)
    output_layer = tf.layers.dense(second_hidden_layer,1)
    
    # Reshape output layer to 1-dim Tensor to return predictions
    predictions = tf.reshape(output_layer,[-1])
    predictions_dict = {"age":predictions}
    
    # Calculate loss using mean squared error
    loss = tf.losses.mean_squared_error(labels, predictions)
    
    # calculate root mean squared error as additional eval metric
    eval_metric_ops = {
        "rmse": tf.metrics.root_mean_squared_error(
        tf.cast(labels, tf.float64), predictions)
    }
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=param["learning_rate"])
    train_op = optimizer.minimize(loss=loss,global_step=tf.train_get_global_step())
    
    # Provide an estimator spec for `ModeKeys.EVAL` and `ModeKeys.TRAIN` modes.
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions = predictions
        loss=loss,
        train_op=train_op
        eval_metric_ops=eval_metric_ops
    )

In [15]:
def main(unused_argv):
  # Load datasets
    abalone_train, abalone_test, abalone_predict = maybe_download(
    FLAGS.train_data, FLAGS.test_data, FLAGS.predict_data)

  # Training examples
    training_set = tf.contrib.learn.datasets.base.load_csv_without_header(
      filename=abalone_train, target_dtype=np.int, features_dtype=np.float64)

  # Test examples
    test_set = tf.contrib.learn.datasets.base.load_csv_without_header(
      filename=abalone_test, target_dtype=np.int, features_dtype=np.float64)

  # Set of 7 examples for which to predict abalone ages
    prediction_set = tf.contrib.learn.datasets.base.load_csv_without_header(
      filename=abalone_predict, target_dtype=np.int, features_dtype=np.float64)
    
    # Set model params
    model_params = {"learning_rate": LEARNING_RATE}
    
    # Instantiate Estimator
    nn = tf.estimator.Estimator(model_fn=model_fn,params=model_params)
    
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x":np.array(training_set.data)},
        y=np.array(training_set.target),
        num_epochs=None,
        shuffle=True)
    
    # train
    nn.train(input_fn=train_input_fn,steps=5000)
    
    # score_accuracy
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x":np.array(test_set.data)},
        y=np.array(test_set.target),
        num_epochs=1,
        shuffle=False)
    
    ev = nn.evaluate(input_fn=test_input_fn)
    print("Loss: %s" % ev["loss"])
    print("Root Mean Squared Error: %s" % ev["rmse"])
    
    # print out predictions
    predict_input_fn=tf.estimator.inputs.numpy_input_fn(
        x={"x":prediction_set.data},
        num_epochs=1,
        shuffle=False)
    predictions = nn.predict(input_fn=predict_input_fn)
    for i, p in enumerate(predictions):
        print("Prediction %s: %s" % (i + 1, p["ages"]))
    

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.register("type", "bool", lambda v: v.lower() == "true")
    parser.add_argument(
      "--train_data", type=str, default="", help="Path to the training data.")
    parser.add_argument(
      "--test_data", type=str, default="", help="Path to the test data.")
    parser.add_argument(
      "--predict_data",
      type=str,
      default="",
      help="Path to the prediction data.")
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)
